# NYC Taxi project

This is an attmept at the Kaggle competition to predict the total ride duration in taxi trips in the New York City. Kaggle has provided with training and test data set.

## 1. Answering the data analytics question

What type of data analytics queston is this? Ans. This is a regression problem to predict the ride duration
What is the measurement of success for the project Ans. If the accuracy of prediction is above 90% the model is considered to be a success.
Can the question be answered from the given data? Ans. Yes, a quick run through the data gives enough evidence to suggest that the question can be solved fro m the given data

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import re #for processing regular expression
import datetime #for datetime operations
import time # to get system time

In [12]:
start = time.time()
train_df = pd.read_csv('/Users/shawnchacko/Desktop/Jupyter/NYC Taxi/train.csv')
end = time.time()
print ("Time taken by this script is {} sec.".format(end-start))
train_df.head(10)

Time taken by this script is 6.98058485985 sec.


,id,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,store_and_fwd_flag,trip_duration
0,id2875421,2,2016-03-14 17:24:55,2016-03-14 17:32:30,1,-73.982155,40.767937,-73.964630,40.765602,N,455
1,id2377394,1,2016-06-12 00:43:35,2016-06-12 00:54:38,1,-73.980415,40.738564,-73.999481,40.731152,N,663
2,id3858529,2,2016-01-19 11:35:24,2016-01-19 12:10:48,1,-73.979027,40.763939,-74.005333,40.710087,N,2124
3,id3504673,2,2016-04-06 19:32:31,2016-04-06 19:39:40,1,-74.010040,40.719971,-74.012268,40.706718,N,429
4,id2181028,2,2016-03-26 13:30:55,2016-03-26 13:38:10,1,-73.973053,40.793209,-73.972923,40.782520,N,435
5,id0801584,2,2016-01-30 22:01:40,2016-01-30 22:09:03,6,-73.982857,40.742195,-73.992081,40.749184,N,443
6,id1813257,1,2016-06-17 22:34:59,2016-06-17 22:40:40,4,-73.969017,40.757839,-73.957405,40.765896,N,341
7,id1324603,2,2016-05-21 07:54:58,2016-05-21 08:20:49,1,-73.969276,40.797779,-73.922470,40.760559,N,1551
8,id1301050,1,2016-05-27 23:12:23,2016-05-27 23:16:38,1,-73.999481,40.738400,-73.985786,40.732815,N,255
9,id0012891,2,2016-03-10 21:45:01,2016-03-10 22:05:26,1,-73.981049,40.744339,-73.973000,40.789989,N,1225


## 2. Tyding up the data

Checking if all columns have the same number of entries. ie if any NaN columns exists

In [13]:
train_df.describe()

,vendor_id,passenger_count,pickup_longitude,pickup_latitude,dropoff_longitude,dropoff_latitude,trip_duration
count,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06,1.458644e+06
mean,1.534950e+00,1.664530e+00,-7.397349e+01,4.075092e+01,-7.397342e+01,4.075180e+01,9.594923e+02
std,4.987772e-01,1.314242e+00,7.090186e-02,3.288119e-02,7.064327e-02,3.589056e-02,5.237432e+03
min,1.000000e+00,0.000000e+00,-1.219333e+02,3.435970e+01,-1.219333e+02,3.218114e+01,1.000000e+00
25%,1.000000e+00,1.000000e+00,-7.399187e+01,4.073735e+01,-7.399133e+01,4.073588e+01,3.970000e+02
50%,2.000000e+00,1.000000e+00,-7.398174e+01,4.075410e+01,-7.397975e+01,4.075452e+01,6.620000e+02
75%,2.000000e+00,2.000000e+00,-7.396733e+01,4.076836e+01,-7.396301e+01,4.076981e+01,1.075000e+03
max,2.000000e+00,9.000000e+00,-6.133553e+01,5.188108e+01,-6.133553e+01,4.392103e+01,3.526282e+06


Looks like all the columns have the same number of entries. But as the number of entries are so huge lets dive deap and see if we can get a clearer picture.

In [37]:
train_df.isnull().sum() #Checking if there is any non zero values in each column

id                    0
vendor_id             0
pickup_datetime       0
dropoff_datetime      0
passenger_count       0
pickup_longitude      0
pickup_latitude       0
dropoff_longitude     0
dropoff_latitude      0
store_and_fwd_flag    0
trip_duration         0
dtype: int64

In [36]:
print ("Number of columns in the data set is {}.".format(train_df.shape[0])) #counting the number of columns
print ("Number of non null values in the data set are {}.".format(train_df.id.nunique())) #counting the number of non null values

Number of columns in the data set is 1458644.
Number of non null values in the data set are 1458644.
